In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
player_data = pd.read_csv('./Datasets/player_game_data.csv')
team_data = pd.read_csv('./Datasets/team_game_data.csv')

In [3]:
print(player_data.isna().sum()[player_data.isna().sum()>0])
print(team_data.isna().sum().sum())

player_name    4
dtype: int64
0


## Effective field goal percentages

$$ EFG\% = \frac{FGM + 0.5 * 3PM}{ FGA} $$

In [4]:
# split team data to regular season and playoff

team_reg = team_data.loc[team_data.gametype == 2, :]
team_playoff = team_data.loc[team_data.gametype == 4, :]

# calculate offensive and defensive totals for counting stats

count_stats = team_data.columns[14:]

team_reg_totals = team_reg.groupby(['season', 'off_team'])[count_stats].sum()
team_reg_def_totals = team_reg.groupby(['season', 'def_team'])[count_stats].sum()
def calc_efg_pct(row):
    efg_pct = (row['fgmade'] + 0.5*row['fg3made'])/row['fgattempted']*100
    return round(efg_pct, 1)

team_reg_totals['eFG'] = team_reg_totals.apply(calc_efg_pct, axis=1)
team_reg_def_totals['eFG'] = team_reg_def_totals.apply(calc_efg_pct, axis=1)

print('GSW offensive eFG percentage in 2015-2016 regular season is', team_reg_totals.loc[(2015, 'GSW'), 'eFG'])
print('GSW defensive eFG percentage in 2015-2016 regular season is', team_reg_def_totals.loc[(2015, 'GSW'), 'eFG'])

GSW offensive eFG percentage in 2015-2016 regular season is 56.3
GSW defensive eFG percentage in 2015-2016 regular season is 47.9


## Win frequency of the team with higher EFG%

In [14]:
# calculate effective field goal percentage of each team in each game

team_reg.loc[:,'eFG'] = team_reg.apply(calc_efg_pct, axis=1)

team_reg.loc[:,'eFG'] = team_reg.apply(calc_efg_pct, axis=1)
# find the diff for points, offensive rebounds and eFG for the offensive team

diff_df = team_reg.groupby('nbagameid')[['points', 'eFG', 'reboffensive']] \
                .apply(lambda x:(x.iloc[0,:] - x.iloc[1,:])) \
                .reset_index() \
                .rename({'points': 'points_diff', 'eFG':'eFG_diff', 'reboffensive':'reboff_diff'}, axis=1) 

# team with better eFG wins if the signs of points_diff and efG_diff are same

higher_eff_win_pct  = diff_df.query('eFG_diff != 0') \
                            .assign(p=lambda x:(x['points_diff']>0) ==  (x['eFG_diff'] > 0)) \
                            .p.mean()
        
print(round(100*higher_eff_win_pct, 1))


81.8


## Win Frequency of teams with more offensive rebounds

In [6]:
# team with better eFG wins if the signs of points_diff and reboff_diff are same
higher_reboff_win_pct  = diff_df.query('reboff_diff != 0') \
                            .assign(p=lambda x:(x['points_diff'] > 0) == (x['reboff_diff'] > 0)) \
                            .p.mean()
        
print(round(100*higher_reboff_win_pct, 1))


46.2


## Percentage of games players are available

In [7]:
# Add flags for whether player played in the game and whether he is available
player_data['played'] = (player_data.seconds > 0)
player_data['available'] = 1 - player_data.missed

# dataframes for regular season and playoff games
player_reg = player_data.loc[player_data.gametype == 2, :]
player_playoff = player_data.loc[player_data.gametype == 4, :]

# calculate regular season totals for games played, points and percentage of available games 
player_availablity = player_reg.groupby(['season', 'player_name']) \
                                .agg(available_pct=('available', 'mean'),
                                     total_points = ('points', 'sum'),
                                     total_played=('played', 'sum'),
                                     )  \
                                .reset_index()
                                
# Add points per game
player_availablity['ppg'] = player_availablity['total_points']/player_availablity['total_played']

# Apply the constraints
available_pct = player_availablity.query('ppg > 25 & available_pct >= 0.25') \
                                   .available_pct \
                                   .mean()

# Calculate the mean available game percentage per season
print(round(100*available_pct, 1))


82.9


## home court team winning percentage in playoffs by round

In [ ]:
# sort by gamedate, to identify first and last games of a playoff series
team_playoff = team_playoff.sort_values(by=['gamedate'])

# keep one row for each game, and keep relevant columns
team_playoff = team_playoff.drop_duplicates(subset='nbagameid') \
                            .loc[:,['season', 'gamedate', 'off_team', 'def_team', 'off_home', 'off_win']]

# create a new column for playoff series name containing team names and year
def create_series_name(row):
    teams = sorted([row['off_team'], row['def_team']])
    return teams[0] + teams[1] + str(row['season'])

team_playoff['series_name'] = team_playoff.apply(create_series_name, axis = 1)

# create a column for the winner team name fof each game
def winner(row):
    if row['off_win'] == 1:
        return row['off_team']
    else:
        return row['def_team']

team_playoff['winner'] = team_playoff.apply(winner, axis = 1)

# use first games to identify home court advantage and rounds
first_games = team_playoff.drop_duplicates('series_name')

# the first 8 games each playoff are round 1, next 4 are round 2, next 2 are conference finals, next one is Finals
rounds_season = ['Round 1']*8 + ['Round 2']*4 + ['Conference Finals']*2 + ['Finals']
first_games['Round'] = 19 * rounds_season

# home side on first game has home court advantage
def home_court_ad(row):
    if row['off_home'] == 1:
        return row['off_team']
    else:
        return row['def_team']

first_games['home_court_advantage'] = first_games.apply(home_court_ad, axis=1)

# summarize info
playoff_summary = first_games[['series_name', 'season', 'off_team', 'def_team','Round', 'home_court_advantage']]

# winnner of last game is series winner
series_winners = team_playoff.drop_duplicates(subset='series_name', keep='last')[['series_name', 'winner']] \
                            .rename({'winner':'series_winner'}, axis = 1)

playoff_summary = pd.merge(playoff_summary, series_winners, on='series_name')

# check whether home team wins
playoff_summary['home_court_ad_wins'] = \
    (playoff_summary['home_court_advantage'] == playoff_summary['series_winner'])




In [15]:
# report average number of wins per round between 2014-2023
playoff_summary.loc[playoff_summary.season.isin(range(2014, 2024)), :] \
                .groupby('Round') \
                .agg(win_pct=('home_court_ad_wins', 'mean'))


,win_pct
Round,
Conference Finals,0.555556
Finals,0.777778
Round 1,0.847222
Round 2,0.638889


### teams with +5 rating in regular season for each season 2014-2021

In [9]:
def calc_rtg(points, possessions):
    rtg = points / possessions
    return round(100*rtg, 2)

team_reg_totals['ORTG'] = team_reg_totals.apply(lambda row:calc_rtg(row['points'], row['possessions']), axis=1)
team_reg_totals['DRTG'] = team_reg_def_totals.apply(lambda row:calc_rtg(row['points'], row['possessions']), axis=1)

team_reg_totals['netRTG'] = team_reg_totals['ORTG'] - team_reg_totals['DRTG']

teams_plus5_net_rtg = team_reg_totals.query('netRTG >= 5')[['netRTG' ]]

### teams that make second round

In [10]:
# get a list of round 2 teams by combining offensive and defensive teams in playoff_summary

round2_teams =pd.concat([playoff_summary[['season', 'off_team', 'Round', 'series_name']].rename({'off_team':'team'}, axis=1),
                        playoff_summary[['season', 'def_team', 'Round', 'series_name']].rename({'def_team':'team'}, axis=1)
                        ], axis=0) \
                        .query('Round == "Round 2"') \
                        .rename({'season':'playoff_season'}, axis=1)
teams_plus5_net_rtg = teams_plus5_net_rtg.reset_index().rename({'off_team':'team'}, axis=1)
teams_plus5_net_rtg['playoff_season'] = teams_plus5_net_rtg['season'] + 1

round2_teams_plus5_net_rtg = pd.merge(round2_teams, 
                                        teams_plus5_net_rtg, 
                                        how='inner', 
                                        on = ['playoff_season', 'team'])

#round2_teams_plus5_net_rtg = round2_teams_plus5_net_rtg.drop(['next_season'], axis=1)

second_round_pct = round(100*len(round2_teams_plus5_net_rtg)/len(teams_plus5_net_rtg), 1)
print(f'Percentage of teams with +5 or more net rating that made the second round \
in the playoffs the following season is {second_round_pct}')

Percentage of teams with +5 or more net rating that made the second round in the playoffs the following season is 53.8


In [11]:
pd.merge(round2_teams, teams_plus5_net_rtg, how='inner', on = ['playoff_season', 'team']).shape[0]
len(teams_plus5_net_rtg)

39

## top 5 players from each team with most minutes in regular season

In [12]:
# filter regular season games
player_min_reg = player_data.loc[(player_data.gametype==2) & (player_data.season.isin(range(2014,2022))), ['season', 'team', 'player_name', 'seconds']]

# calculate total for time played
player_min_totals = player_min_reg.groupby(['season', 'team', 'player_name']) \
                                .agg(total_secs = ('seconds', 'sum'))  \
                                .reset_index()

# get top 5 players for each season team combination                                
player_min_top_5 = player_min_totals.sort_values('total_secs', ascending=False) \
                                .groupby(['season', 'team']) \
                                .head(5) \
                                .sort_values(by=['season', 'team'])

# create column to merge with playoff data
player_min_top_5['playoff_season'] = player_min_top_5['season'] + 1

# add the following year's playoff series information
player_min_top_5 = pd.merge(player_min_top_5, 
                            round2_teams_plus5_net_rtg, 
                            on=['playoff_season', 'team']
                            )
                

# player_min_totals.set_index(['player_name']) \
#                     .groupby(['season', 'team']) \
#                     ['total_secs'] \
#                     .nlargest(5) \
#                     .reset_index()

# player_min_totals.groupby(['season', 'team']) \
#                     .apply(lambda x:x[['player_name', 'total_secs']].sort_values('total_secs', ascending=False).head(5)) \
#                     .reset_index() \
#                     .drop('level_2', axis=1)

## round 2 players

In [13]:
# create a new column for playoff series name containing team names and year
player_playoff = player_playoff.rename({'team':'off_team', 'opp_team':'def_team'}, axis=1)

player_playoff['series_name'] = player_playoff.apply(create_series_name, axis = 1)
# Find players who played in the playoffs, with total seconds > 0

player_playoff = player_playoff[['player_name', 'seconds', 'series_name', 'off_team']] \
                        .groupby(['series_name',  'player_name', 'off_team']) \
                        .agg(total_time = ('seconds', 'sum')) \
                        .reset_index() \
                        .query('total_time>0') \
                        .rename({'off_team':'team'}, axis=1)

qualified_players = pd.merge(player_playoff, player_min_top_5, on=['series_name', 'player_name', 'team'])  \
                        .sort_values(['series_name'])

qualified_players_pct = round(100*len(qualified_players)/len(player_min_top_5), 1)
print(f'Percentage of players with top five minutes who played in round 2 of playoffs next year is {qualified_players_pct}')

Percentage of players with top five minutes who played in round 2 of playoffs next year is 78.1
